# Import Libraries 

In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
import shutil  # Manage directories
import sys  # Read system parameters.

import keras  # Provide a frontend for TensorFlow.
import matplotlib as mpl  # Create 2D charts.
import matplotlib.pyplot as plt
import numpy as np  # Work with multi-dimensional arrays and matrices.
import sklearn  # Perform data mining and analysis.
import tensorflow  # Train neural networks for deep learning.
from keras import datasets
from numpy.random import seed

# Summarize software libraries used.
print("Libraries used in this project:")
print("- Python {}".format(sys.version))
print("- NumPy {}".format(np.__version__))
print("- Matplotlib {}".format(mpl.__version__))
print("- scikit-learn {}".format(sklearn.__version__))
print("- TensorFlow {}".format(tensorflow.__version__))
print("- Keras {}\n".format(keras.__version__))

Libraries used in this project:
- Python 3.10.13 (main, Sep 11 2023, 08:21:04) [Clang 14.0.6 ]
- NumPy 1.26.4
- Matplotlib 3.9.2
- scikit-learn 1.5.1
- TensorFlow 2.10.0
- Keras 2.10.0



# Load the Dataset

In [2]:
(X_train, y_train), (X_test, y_test) = datasets.fashion_mnist.load_data()
print("Loaded {} fashion training records.".format(len(X_train.data)))
print("Loaded {} fashion test records.".format(len(X_test.data)))

Loaded 60000 fashion training records.
Loaded 10000 fashion test records.


**Spotlights** 

The Fashion-MNIST dataset was imported using Keras, which serves as a user-friendly interface for the TensorFlow deep learning library. It consists of numerical data that can be transformed into small grayscale images representing various clothing items. The training dataset comprised 60,000 records, while the test dataset included 10,000 records.

# Explore the Data

In [7]:
print("THe shape of the feature space is: {}\n".format(X_train[0].shape))

print("Some examples:\n")
print(X_train[6:9])

THe shape of the feature space is: (28, 28)

Some examples:

[[[  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  ...
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]]

 [[  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  ...
  [  0   0   0 ... 180   0   0]
  [  0   0   0 ...  72   0   0]
  [  0   0   0 ...  70   0   0]]

 [[  0   0   0 ...   0   0   0]
  [  0   0   0 ...   0   0   0]
  [  0   0   0 ...  39   1   0]
  ...
  [  0   0   0 ... 238   0   0]
  [  0   0   0 ... 131   0   0]
  [  0   0   0 ...   0   0   0]]]


**Spotlights** 

- The structure of the feature space indicates that the training set is multi-dimensional; instead of having just 28 features, each image actually consists of 28 × 28 features, reflecting its dimensions of 28 pixels in width and 28 pixels in height.
- Features for example images 6, 7, and 8 are displayed, showing that while many values are truncated, the majority are 0, with a few having positive values. Each number signifies the intensity of a specific pixel in grayscale. Essentially, a value of 0 represents complete black, while 255 indicates complete white.
- This multi-dimensional representation allows for a more detailed analysis of the images, capturing the nuances of pixel intensity across the entire image space.

# Examine Class Names

In [8]:
class_names = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

for i in range(10):
    print("{} ({})".format(class_names[i], np.unique(y_train)[i]))

T-shirt/top (0)
Trouser (1)
Pullover (2)
Dress (3)
Coat (4)
Sandal (5)
Shirt (6)
Sneaker (7)
Bag (8)
Ankle boot (9)


**Spotlights**

Each class label corresponds to its specific class name, which represents the type of clothing. In total, there are 10 distinct classes.